In [0]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-03-30 20:19:18--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K   435KB/s    in 2.1s    

2020-03-30 20:19:21 (435 KB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Homework_week_22b").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
# url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

user_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"), sep='\t', header=True)

# Show DataFrame
user_data_df.show()
user_data_df.count()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

904765

In [5]:
# Drop null values
user_data_df = user_data_df.dropna()
user_data_df.count()

904663

In [6]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for only columns with active users
cleaned_df = user_data_df.filter(col("verified_purchase")  == "Y")
cleaned_df = cleaned_df.filter(col("helpful_votes")  > 0)
cleaned_df = cleaned_df.filter(col("total_votes")  > 5)
print("Total Number of Musical Instrument Reviews to be analyzed: ", cleaned_df.count())
cleaned_df.show()

Total Number of Musical Instrument Reviews to be analyzed:  55314
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21380993| RPZHAM1T3RMDU|B005EHILV4|     820471122|BEHRINGER XENYX 3...|Musical Instruments|          5|            6|          7|   N|                Y|  Great Little Mixer|Great little mixe...| 2015-08-31|
|         US|   28770559|R2TGT0CDTCAAHW|B0

In [7]:
# Create user dataframe to find count of reviews per customer (for table Customers )
df_customers = cleaned_df.groupby('customer_id').agg({'review_id': 'count'})
df_customers = df_customers.withColumnRenamed('count(review_id)','customer_count')
print("Total number of Customers: ", df_customers.count())
df_customers.show()

Total number of Customers:  49196
+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   39705765|             1|
|   45365471|             1|
|   51653295|             1|
|   51798805|             1|
|   18925624|             1|
|   15583348|             1|
|   51068983|             1|
|   14824990|             1|
|   37030588|             1|
|   45873136|             1|
|   14784362|             2|
|   13938576|             1|
|   28199550|             1|
|   50973182|             1|
|   12374662|             1|
|   13931753|             1|
|   12894296|             1|
|   43331905|             1|
|   12193166|             1|
|   36232236|             1|
+-----------+--------------+
only showing top 20 rows



In [8]:
df_customers = df_customers.selectExpr("cast(customer_id as int) customer_id","cast(customer_count as int) customer_count")
df_customers.dtypes

[('customer_id', 'int'), ('customer_count', 'int')]

In [9]:
# Create user dataframe to store product Information
df_products = cleaned_df.select(['Product_ID','product_title','product_category']).distinct()
print("Total number of Products: ", df_products.count())
df_products.show()

Total number of Products:  20300
+----------+--------------------+-------------------+
|Product_ID|       product_title|   product_category|
+----------+--------------------+-------------------+
|B00HVLUQW8|Audio-Technica AT...|Musical Instruments|
|B00EOYKGH0|Cecilio CVN-300 S...|Musical Instruments|
|B00ZY33B40|Zoom TAC-2R Two-c...|Musical Instruments|
|B007R0ORR8|Suzuki Musical In...|Musical Instruments|
|B00DJ66G3W|Yamaha 61-Key Por...|Musical Instruments|
|B000G3FNVM|Numark TTUSB | 33...|Musical Instruments|
|B00PRTXKVA|Crescent 4/4 Begi...|Musical Instruments|
|B000V6GB24|K & M Mic stand w...|Musical Instruments|
|B00OLAJ6YQ|Glory Closed Hole...|Musical Instruments|
|B00OLAJ3W6|Glory Closed Hole...|Musical Instruments|
|B00I4IPDUW|Yamaha Electronic...|Musical Instruments|
|B000NE3JZ6|D'Addario Kaplan ...|Musical Instruments|
|B00B2WZ8MK|MUSYSIC Professio...|Musical Instruments|
|B00466HM28|Akai Professional...|Musical Instruments|
|B0002CZUTM|Epiphone LP-100 L...|Musical Instrume

In [10]:
df_products.dtypes

[('Product_ID', 'string'),
 ('product_title', 'string'),
 ('product_category', 'string')]

In [11]:
# Create user dataframe for review_id_table
df_review_id_table = cleaned_df.select(["review_id", "customer_id","product_id","product_parent","review_date"])
print("Total number of Reviews: ", df_review_id_table.count())
df_review_id_table.show()

Total number of Reviews:  55314
+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RPZHAM1T3RMDU|   21380993|B005EHILV4|     820471122| 2015-08-31|
|R2TGT0CDTCAAHW|   28770559|B00INJ7HBK|     157027184| 2015-08-31|
|R12DGW3G8CPCCF|    7452185|B00DJZ1OK8|     445844821| 2015-08-31|
|R2WDZPS4FXEYKW|   32672319|B00D2TZKIS|     517123763| 2015-08-31|
|R1H3ZFTQT3TMEN|   34464133|B00GRPIUEU|     169966819| 2015-08-31|
|R3B9PY8CO0U86Q|   42085622|B00U1XK5Z6|     930307093| 2015-08-31|
| RUT7B85ZA1UTZ|   13996084|B002AKKO72|     656698662| 2015-08-31|
|R1Z8J425HTLRRP|   51726804|B004HZGR5Q|     701464811| 2015-08-31|
|R1P8HZW25TLBOL|   38173315|B00MIXFBKG|      21180708| 2015-08-31|
|R29TWRL81U84PR|   14780931|B001LG7TUG|     507028013| 2015-08-31|
| RXTWUZLXBYA3R|   51193249|B00VYJJ1YM|     124812654| 2015-08-31|
|R2ENV9NT1PLDY3|   12573285|B0

In [12]:
df_review_id_table = df_review_id_table.selectExpr("cast(review_id as string) review_id","cast(customer_id as int) customer_id","cast(product_id as string) product_id","cast(product_parent as int) product_parent","cast(review_date as date) review_date")
df_review_id_table.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

In [13]:
df_review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RPZHAM1T3RMDU|   21380993|B005EHILV4|     820471122| 2015-08-31|
|R2TGT0CDTCAAHW|   28770559|B00INJ7HBK|     157027184| 2015-08-31|
|R12DGW3G8CPCCF|    7452185|B00DJZ1OK8|     445844821| 2015-08-31|
|R2WDZPS4FXEYKW|   32672319|B00D2TZKIS|     517123763| 2015-08-31|
|R1H3ZFTQT3TMEN|   34464133|B00GRPIUEU|     169966819| 2015-08-31|
|R3B9PY8CO0U86Q|   42085622|B00U1XK5Z6|     930307093| 2015-08-31|
| RUT7B85ZA1UTZ|   13996084|B002AKKO72|     656698662| 2015-08-31|
|R1Z8J425HTLRRP|   51726804|B004HZGR5Q|     701464811| 2015-08-31|
|R1P8HZW25TLBOL|   38173315|B00MIXFBKG|      21180708| 2015-08-31|
|R29TWRL81U84PR|   14780931|B001LG7TUG|     507028013| 2015-08-31|
| RXTWUZLXBYA3R|   51193249|B00VYJJ1YM|     124812654| 2015-08-31|
|R2ENV9NT1PLDY3|   12573285|B00QLWZMVS|     804312692| 2015-08

In [14]:
# Create user dataframe for review_id_table
df_vine_table = cleaned_df.select(["review_id", "star_rating","helpful_votes","total_votes","vine"])
print("Total number of Vine Records: ", df_vine_table.count())
df_vine_table.show()

Total number of Vine Records:  55314
+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RPZHAM1T3RMDU|          5|            6|          7|   N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|
|R12DGW3G8CPCCF|          1|            4|         15|   N|
|R2WDZPS4FXEYKW|          5|            6|          6|   N|
|R1H3ZFTQT3TMEN|          4|            4|          6|   N|
|R3B9PY8CO0U86Q|          5|            6|          8|   N|
| RUT7B85ZA1UTZ|          5|            8|          9|   N|
|R1Z8J425HTLRRP|          5|            7|          7|   N|
|R1P8HZW25TLBOL|          1|           12|         14|   N|
|R29TWRL81U84PR|          5|           17|         19|   N|
| RXTWUZLXBYA3R|          1|            6|          7|   N|
|R2ENV9NT1PLDY3|          5|           17|         18|   N|
|R39QE4FJRDTRPI|          1|            5|          6|   N|
| R

In [15]:
df_vine_table = df_vine_table.selectExpr("cast(review_id as string) review_id","cast(star_rating as int) star_rating","cast(helpful_votes as int) helpful_votes","cast(total_votes as int) total_votes","cast(vine as string) vine")
df_vine_table.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string')]

Postgres Setup

In [0]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://amazon-review-avr.cn0fs69uwn5x.us-east-1.rds.amazonaws.com:5432/amazon_review_db"
config = {"user":"root", 
          "password": "Welcome!", 
          "driver":"org.postgresql.Driver"}


In [0]:
# Write DataFrame to customer table in RDS

df_customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [0]:
# Write dataframe to products table in RDS

df_products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [0]:
# Write dataframe to review_id table in RDS

df_review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [0]:
# Write dataframe to vine table in RDS

df_vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)